In [59]:
import pandas as pd
import numpy as np
import tensorflow

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-06-04 13:31:23.666423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Every YouTube video is represented by a thumbnail, a small image that, along with the title and channel, serves as the “cover” of the video. Thumbnails that are interest- ing and well-framed attract viewers, while those that are confusing and low-quality encourage viewers to click else- where. As a testament to the important of a good thumb- nail, 90% of the most successful YouTube videos have cus- tom thumbnails [2]. YouTube uploaders without the time or skills to create a custom thumbnail, however, must pick one of 3 frames automatically chosen from the video. Our mission is to improve this frame selection process and help uploaders select high quality frames that will attract viewers to their channel.

http://cs231n.stanford.edu/reports/2017/pdfs/710.pdf
[2] YouTube Creator Academy. Lesson: Make click- able thumbnails. https://creatoracademy. youtube.com/page/lesson/thumbnails# yt-creators-strategies-5.

In [116]:
df = pd.read_csv("../Data/CAvideos.csv")

In [118]:
! pip install tqdm

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)


In [121]:
# load thumbnail and save picture path in df
import urllib.request
from tqdm import tqdm
import os

df_w_jgp = df.copy()

def load_and_save_thumbnail(url, iterator):
    try:

        # get unique thumbnail id 
        splitted_words = url.split("/")
        thumbnail_id = splitted_words[4]

        # create filename to save thumbnail
        file_path = f"../Data/thumbnails/{thumbnail_id}.jpg"
        df_w_jgp.at[iterator,"thumbnail_link"] = file_path

        # check if thumbnail is already downloaded
        if os.path.exists(file_path):
            pass
        else:
            # download and save thumbnail 
            urllib.request.urlretrieve(url, file_path)

    except urllib.error.URLError as e:

        df_w_jgp.at[iterator,"thumbnail_link"] = np.NAN
        # print(f"An error occurred: {e}")
        return None

for i in tqdm(range (0, len(df))):

    # get thumbnail url
    url = df.iloc[i]["thumbnail_link"]

    # safe thumnail and write path into df
    load_and_save_thumbnail(url, i)  

100%|██████████| 40881/40881 [51:43<00:00, 13.17it/s]  


In [ ]:
# Save final df as csv
csv_path = "../Data/dataset_w_jpg.csv"
df_w_jgp.to_csv(csv_path, index=False)

In [ ]:
df_w_jgp

In [6]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import urllib.parse as p
import re
import os
import pickle

SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [7]:
# https://www.thepythoncode.com/article/using-youtube-api-in-python#google_vignette

def youtube_authenticate():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secret_883243427026-gmpvuoto8cd0gsl6djl80sesaa6b5jo0.apps.googleusercontent.com.json"
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    return build(api_service_name, api_version, credentials=creds)

# authenticate to YouTube API
youtube = youtube_authenticate()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=883243427026-gmpvuoto8cd0gsl6djl80sesaa6b5jo0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50636%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=NrXAzAtLlszX4joayhvEnjIHPFtrYx&access_type=offline


In [55]:
# API Documentation https://developers.google.com/youtube/v3/docs/subscriptions/list?hl=de&apix=true

def get_channel_details(youtube, **kwargs):

    request = youtube.channels().list(
        part="statistics", # just get stats
        **kwargs
    )
    response = request.execute()

    return response

In [ ]:
def get_channel_id(youtube, **kwargs):

    request = youtube.channels().list(
        part="statistics", # just get stats
        **kwargs
    )
    response = request.execute()

    return response

In [56]:
channel_name =

video_id = get_channel_id(channel_name) #"UC20vb-R_px4CguHzzBPhoyQ"

# make API call to get channel infos
response = get_channel_details(youtube, id=video_id)
items = response.get("items")[0]
statistics = items["statistics"]

# get stats infos
subscriberCount = statistics["subscriberCount"]
print(f"Subscriber Count: {subscriberCount} ")

Subscriber Count: 24800000 


In [15]:
def print_video_infos(video_response):
    items = video_response.get("items")[0]
    print(items)
    # get the snippet, statistics & content details from the video response
    snippet         = items["snippet"]
    statistics      = items["statistics"]
    content_details = items["contentDetails"]
    # get infos from the snippet
    channel_title = snippet["channelTitle"]
    title         = snippet["title"]
    description   = snippet["description"]
    publish_time  = snippet["publishedAt"]
    # get stats infos
    comment_count = statistics["commentCount"]
    like_count    = statistics["likeCount"]
    view_count    = statistics["viewCount"]
    # get duration from content details
    duration = content_details["duration"]
    # duration in the form of something like 'PT5H50M15S'
    # parsing it to be something like '5:50:15'
    parsed_duration = re.search(f"PT(\d+H)?(\d+M)?(\d+S)", duration).groups()
    duration_str = ""
    for d in parsed_duration:
        if d:
            duration_str += f"{d[:-1]}:"
    duration_str = duration_str.strip(":")
    print(f"""\
    Title: {title}
    Description: {description}
    Channel Title: {channel_title}
    Publish time: {publish_time}
    Duration: {duration_str}
    Number of comments: {comment_count}
    Number of likes: {like_count}
    Number of views: {view_count}
    """)